In [1]:
#Exploring the concept of taking two data frames from two seperate queries and applying RITA calculations


In [2]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
es = Elasticsearch(['https://192.168.1.181:9200'],                  
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')
es.info()

ObjectApiResponse({'name': 'voodoo-onion', 'cluster_name': 'voodoo-onion', 'cluster_uuid': 'dP1VycCSTQ2EPeCVxQ25GA', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [4]:
#query_since_ts = 0
#body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":' + str(query_since_ts) + '}}}]}},"size=10000":[{"@timstamp":{"order":"asc"}}]}'
#body_filter = '{"query":{"@timestamp":{}},"size=10000":[{"@timestamp":{"order":"desc"}}]}'

body_filter = '{"query":{"bool":{"must":[],"filter":[{"range":{"@timestamp":{"format":"strict_date_optional_time","gte":"2021-12-30","lte":"2022-01-01"}}}]}'


In [5]:

#Firewall 192.168.1.154

#s1 = searchContext.query('query_string', query='(event.dataset:(conn or http) AND network.transport:tcp) AND destination.port:(*) AND (@timestamp > "now-1d/d") AND NOT source.ip:192.168.1.154')
s1 = searchContext.query('query_string', query='(destination.port:(*) AND NOT source.ip:192.168.1.154')

s2 = searchContext.query('query_string', query='http.virtual_host:*')

In [6]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


#base response query
#response = s.execute()
#if response.success():
#  df = pd.DataFrame((d.to_dict() for d in s.scan()))
#json_struct = json.loads(df.to_json(orient="records"))
#df_flat = pd.io.json.json_normalize(json_struct)
#df_flat.head(20)

#response = s.execute()
#if response.success():
    #df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})
#display(df)
#df.info()

In [7]:
#data frame 1 for s1 which is search query 1
s1 = searchContext.query('query_string', query='(event.dataset:(conn or http) AND network.transport:tcp) AND destination.port:(53 OR 80 OR 8080 OR 443) AND (@timestamp > "now-1d/d") AND NOT source.ip:192.168.1.154')


response = s1.execute()
df1 = pd.DataFrame(columns=['ts', 'source.ip','destination.ip','virtual_host','destination.port'])
if response.success():
    for d in s1[:10000]:
        try:
            df1 = df1.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'virtual_host': d['http']['virtual_host'], 'destination.port': d['destination']['port'], 'http.method': d['http']['method']}, ignore_index=True)
        except KeyError:
            pass
            

display(df1)

df1.info(5)

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2022-01-03T23:08:15.924Z,192.168.128.11,209.197.3.8,ctldl.windowsupdate.com,80,GET
1,2022-01-03T23:05:37.037Z,192.168.128.10,209.197.3.8,ctldl.windowsupdate.com,80,GET
2,2022-01-03T22:18:56.379Z,192.168.128.50,8.240.195.126,ctldl.windowsupdate.com,80,GET
3,2022-01-03T21:50:13.249Z,192.168.128.11,23.66.101.136,ctldl.windowsupdate.com,80,GET
4,2022-01-03T21:26:53.468Z,192.168.128.26,8.240.196.126,ctldl.windowsupdate.com,80,GET
...,...,...,...,...,...,...
1674,2022-01-03T00:14:58.028Z,192.168.128.11,72.21.81.240,download.windowsupdate.com,80,GET
1675,2022-01-03T00:14:57.970Z,192.168.128.11,72.21.81.240,download.windowsupdate.com,80,GET
1676,2022-01-03T00:11:00.018Z,192.168.128.50,192.168.120.158,192.168.120.158,80,GET
1677,2022-01-03T00:10:29.107Z,192.168.128.50,184.85.161.126,crl.disa.mil,80,GET


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679 entries, 0 to 1678
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                1679 non-null   object
 1   source.ip         1679 non-null   object
 2   destination.ip    1679 non-null   object
 3   virtual_host      1679 non-null   object
 4   destination.port  1679 non-null   object
 5   http.method       1679 non-null   object
dtypes: object(6)
memory usage: 78.8+ KB


In [8]:
df1['ts'] = pd.to_datetime(df1['ts'],infer_datetime_format=True)
df1.head(10)

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2022-01-03 23:08:15.924000+00:00,192.168.128.11,209.197.3.8,ctldl.windowsupdate.com,80,GET
1,2022-01-03 23:05:37.037000+00:00,192.168.128.10,209.197.3.8,ctldl.windowsupdate.com,80,GET
2,2022-01-03 22:18:56.379000+00:00,192.168.128.50,8.240.195.126,ctldl.windowsupdate.com,80,GET
3,2022-01-03 21:50:13.249000+00:00,192.168.128.11,23.66.101.136,ctldl.windowsupdate.com,80,GET
4,2022-01-03 21:26:53.468000+00:00,192.168.128.26,8.240.196.126,ctldl.windowsupdate.com,80,GET
5,2022-01-03 20:40:04.569000+00:00,192.168.128.11,8.240.201.126,ctldl.windowsupdate.com,80,GET
6,2022-01-03 20:09:29.256000+00:00,192.168.128.7,23.75.199.76,crl.disa.mil,80,GET
7,2022-01-03 20:09:29.184000+00:00,192.168.128.7,156.112.101.142,ocsp.disa.mil,80,GET
8,2022-01-03 20:09:20.662000+00:00,192.168.128.11,23.75.199.76,crl.disa.mil,80,GET
9,2022-01-03 20:08:41.367000+00:00,192.168.128.50,23.75.199.76,crl.disa.mil,80,GET


In [9]:
#data frame 2 for s2 which is search query 2
s2 = searchContext.query('query_string', query='http.virtual_host:*')

response = s2.execute()
df2 = pd.DataFrame(columns=['ts', 'source.ip','destination.ip','virtual_host','destination.port'])
if response.success():
    for d in s2[:10000]:
        try:
            df2 = df2.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'virtual_host': d['http']['virtual_host'], 'destination.port': d['destination']['port'], 'http.method': d['http']['method']}, ignore_index=True)
        except KeyError:
            pass
            

display(df2)

df2.info(5)

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2021-12-29T22:20:53.783Z,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
1,2021-12-29T22:20:52.636Z,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
2,2021-12-29T22:20:53.427Z,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
3,2021-12-29T22:20:53.473Z,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
4,2021-12-29T22:20:55.345Z,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
...,...,...,...,...,...,...
9995,2022-01-04T10:18:18.539Z,192.168.128.10,104.84.231.160,download.windowsupdate.com,80,GET
9996,2022-01-04T10:18:18.527Z,192.168.1.154,104.84.231.160,download.windowsupdate.com,80,GET
9997,2022-01-04T10:18:18.527Z,192.168.128.10,104.84.231.160,download.windowsupdate.com,80,GET
9998,2022-01-04T10:18:16.664Z,192.168.1.154,104.84.231.160,download.windowsupdate.com,80,GET


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                10000 non-null  object
 1   source.ip         10000 non-null  object
 2   destination.ip    10000 non-null  object
 3   virtual_host      10000 non-null  object
 4   destination.port  10000 non-null  object
 5   http.method       10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB


In [10]:
df2['ts'] = pd.to_datetime(df2['ts'],infer_datetime_format=True)
df2.head(10)

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2021-12-29 22:20:53.783000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
1,2021-12-29 22:20:52.636000+00:00,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
2,2021-12-29 22:20:53.427000+00:00,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
3,2021-12-29 22:20:53.473000+00:00,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
4,2021-12-29 22:20:55.345000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
5,2021-12-29 22:20:53.066000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
6,2021-12-29 22:20:54.008000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
7,2021-12-29 22:20:53.972000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
8,2021-12-29 22:20:55.384000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
9,2021-12-29 22:20:55.468000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET


In [11]:
# Merge the two to create our final dataframe
df = pd.concat([df1, df2], ignore_index=True)
print(df.head(5))
print(df.tail(5))

                                ts       source.ip destination.ip  \
0 2022-01-03 23:08:15.924000+00:00  192.168.128.11    209.197.3.8   
1 2022-01-03 23:05:37.037000+00:00  192.168.128.10    209.197.3.8   
2 2022-01-03 22:18:56.379000+00:00  192.168.128.50  8.240.195.126   
3 2022-01-03 21:50:13.249000+00:00  192.168.128.11  23.66.101.136   
4 2022-01-03 21:26:53.468000+00:00  192.168.128.26  8.240.196.126   

              virtual_host destination.port http.method  
0  ctldl.windowsupdate.com               80         GET  
1  ctldl.windowsupdate.com               80         GET  
2  ctldl.windowsupdate.com               80         GET  
3  ctldl.windowsupdate.com               80         GET  
4  ctldl.windowsupdate.com               80         GET  
                                    ts       source.ip  destination.ip  \
11674 2022-01-04 10:18:18.539000+00:00  192.168.128.10  104.84.231.160   
11675 2022-01-04 10:18:18.527000+00:00   192.168.1.154  104.84.231.160   
11676 2022-01-0

In [12]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'source.ip'
f_dst_ip = 'destination.ip'
f_dst_host = 'virtual_host'
f_dst_port = 'destination.port'
f_http_method = 'http.method'
f_delimiter = '\t'

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_host,f_http_method, f_dst_port,'deltas']

In [13]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(5)

ts
source.ip      destination.ip virtual_host                       destination.port http.method                                                                                                                                                                                                                                                                                  
162.142.125.58 192.168.1.154  99.62.119.209                      1195             GET                                                                                                                                                                                                                                                        [2021-12-31 15:05:50.792000+00:00]
192.168.1.154  104.100.128.70 tile-service.weather.microsoft.com 80               GET                                            [2021-12-30 12:54:02.017000+00:00, 2022-01-03 12:26:29.951000+00:00, 2022-01-03 12:26:29.687000+00:00, 2022-01-04 11:29:09.448000+00:00, 2022-01-04 11:22:29.361000+00:00, 2021-12-31 13:54:15.480000+00:00, 2022-01-03 12:26:53.163000+00:00]
               104.100.149.44 go.microsoft.com                   80               GET                                                                                                                                                  [2021-12-30 11:50:30.409000+00:00, 2022-01-01 11:50:27.451000+00:00, 2022-01-03 12:30:03.016000+00:00, 2022-01-03 12:19:50.114000+00:00]
                                                                                  POST                                                                                                               [2022-01-01 07:36:45.656000+00:00, 2022-01-01 07:36:45.452000+00:00, 2022-01-01 07:36:44.767000+00:00, 2022-01-01 07:36:43.911000+00:00, 2022-01-01 07:36:42.408000+00:00]
               104.100.156.12 crl.disa.mil                       80               GET          [2022-01-03 08:30:19.681000+00:00, 2022-01-04 12:09:23.863000+00:00, 2021-12-30 08:29:59.003000+00:00, 2021-12-31 08:32:21.772000+00:00, 2022-01-01 09:45:35.958000+00:00, 2022-01-02 09:45:45.477000+00:00, 2022-01-03 08:32:37.617000+00:00, 2022-01-04 12:08:45.682000+00:00]

In [14]:
df.reset_index(inplace=True)
df.head(5)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts
0,162.142.125.58,192.168.1.154,99.62.119.209,1195,GET,[2021-12-31 15:05:50.792000+00:00]
1,192.168.1.154,104.100.128.70,tile-service.weather.microsoft.com,80,GET,"[2021-12-30 12:54:02.017000+00:00, 2022-01-03 12:26:29.951000+00:00, 2022-01-03 12:26:29.687000+00:00, 2022-01-04 11:29:09.448000+00:00, 2022-01-04 11:22:29.361000+00:00, 2021-12-31 13:54:15.480000+00:00, 2022-01-03 12:26:53.163000+00:00]"
2,192.168.1.154,104.100.149.44,go.microsoft.com,80,GET,"[2021-12-30 11:50:30.409000+00:00, 2022-01-01 11:50:27.451000+00:00, 2022-01-03 12:30:03.016000+00:00, 2022-01-03 12:19:50.114000+00:00]"
3,192.168.1.154,104.100.149.44,go.microsoft.com,80,POST,"[2022-01-01 07:36:45.656000+00:00, 2022-01-01 07:36:45.452000+00:00, 2022-01-01 07:36:44.767000+00:00, 2022-01-01 07:36:43.911000+00:00, 2022-01-01 07:36:42.408000+00:00]"
4,192.168.1.154,104.100.156.12,crl.disa.mil,80,GET,"[2022-01-03 08:30:19.681000+00:00, 2022-01-04 12:09:23.863000+00:00, 2021-12-30 08:29:59.003000+00:00, 2021-12-31 08:32:21.772000+00:00, 2022-01-01 09:45:35.958000+00:00, 2022-01-02 09:45:45.477000+00:00, 2022-01-03 08:32:37.617000+00:00, 2022-01-04 12:08:45.682000+00:00]"


In [15]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count
0,162.142.125.58,192.168.1.154,99.62.119.209,1195,GET,[2021-12-31 15:05:50.792000+00:00],1
1,192.168.1.154,104.100.128.70,tile-service.weather.microsoft.com,80,GET,"[2021-12-30 12:54:02.017000+00:00, 2022-01-03 12:26:29.951000+00:00, 2022-01-03 12:26:29.687000+00:00, 2022-01-04 11:29:09.448000+00:00, 2022-01-04 11:22:29.361000+00:00, 2021-12-31 13:54:15.480000+00:00, 2022-01-03 12:26:53.163000+00:00]",7


In [16]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 2]
df.shape

(335, 7)

In [17]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count
1,192.168.1.154,104.100.128.70,tile-service.weather.microsoft.com,80,GET,"[2021-12-30 12:54:02.017000+00:00, 2021-12-31 13:54:15.480000+00:00, 2022-01-03 12:26:29.687000+00:00, 2022-01-03 12:26:29.951000+00:00, 2022-01-03 12:26:53.163000+00:00, 2022-01-04 11:22:29.361000+00:00, 2022-01-04 11:29:09.448000+00:00]",7
2,192.168.1.154,104.100.149.44,go.microsoft.com,80,GET,"[2021-12-30 11:50:30.409000+00:00, 2022-01-01 11:50:27.451000+00:00, 2022-01-03 12:19:50.114000+00:00, 2022-01-03 12:30:03.016000+00:00]",4
3,192.168.1.154,104.100.149.44,go.microsoft.com,80,POST,"[2022-01-01 07:36:42.408000+00:00, 2022-01-01 07:36:43.911000+00:00, 2022-01-01 07:36:44.767000+00:00, 2022-01-01 07:36:45.452000+00:00, 2022-01-01 07:36:45.656000+00:00]",5
4,192.168.1.154,104.100.156.12,crl.disa.mil,80,GET,"[2021-12-30 08:29:59.003000+00:00, 2021-12-31 08:32:21.772000+00:00, 2022-01-01 09:45:35.958000+00:00, 2022-01-02 09:45:45.477000+00:00, 2022-01-03 08:30:19.681000+00:00, 2022-01-03 08:32:37.617000+00:00, 2022-01-04 12:08:45.682000+00:00, 2022-01-04 12:09:23.863000+00:00]",8
5,192.168.1.154,104.102.249.16,ctldl.windowsupdate.com,80,GET,"[2021-12-30 02:48:48.943000+00:00, 2021-12-30 02:48:59.056000+00:00, 2022-01-02 02:08:59.635000+00:00]",3


In [18]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas
1,192.168.1.154,104.100.128.70,tile-service.weather.microsoft.com,80,GET,"[2021-12-30 12:54:02.017000+00:00, 2021-12-31 13:54:15.480000+00:00, 2022-01-03 12:26:29.687000+00:00, 2022-01-03 12:26:29.951000+00:00, 2022-01-03 12:26:53.163000+00:00, 2022-01-04 11:22:29.361000+00:00, 2022-01-04 11:29:09.448000+00:00]",7,"[3613.0, 81134.0, 0.0, 23.0, 82536.0, 400.0]"
2,192.168.1.154,104.100.149.44,go.microsoft.com,80,GET,"[2021-12-30 11:50:30.409000+00:00, 2022-01-01 11:50:27.451000+00:00, 2022-01-03 12:19:50.114000+00:00, 2022-01-03 12:30:03.016000+00:00]",4,"[86397.0, 1762.0, 612.0]"


In [19]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 60)

In [20]:
df.head(3)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv
1,192.168.1.154,104.100.128.70,tile-service.weather.microsoft.com,80,GET,"[2021-12-30 12:54:02.017000+00:00, 2021-12-31 13:54:15.480000+00:00, 2022-01-03 12:26:29.687000+00:00, 2022-01-03 12:26:29.951000+00:00, 2022-01-03 12:26:53.163000+00:00, 2022-01-04 11:22:29.361000+00:00, 2022-01-04 11:29:09.448000+00:00]",7,"[3613.0, 81134.0, 0.0, 23.0, 82536.0, 400.0]",23.0,2006.5,81134.0,77144.0,81111.0,0.951092,1995.0,1355.116667
2,192.168.1.154,104.100.149.44,go.microsoft.com,80,GET,"[2021-12-30 11:50:30.409000+00:00, 2022-01-01 11:50:27.451000+00:00, 2022-01-03 12:19:50.114000+00:00, 2022-01-03 12:30:03.016000+00:00]",4,"[86397.0, 1762.0, 612.0]",1072.0,1762.0,52543.0,50091.0,51471.0,0.973189,1150.0,39.533333
3,192.168.1.154,104.100.149.44,go.microsoft.com,80,POST,"[2022-01-01 07:36:42.408000+00:00, 2022-01-01 07:36:43.911000+00:00, 2022-01-01 07:36:44.767000+00:00, 2022-01-01 07:36:45.452000+00:00, 2022-01-01 07:36:45.656000+00:00]",5,"[1.0, 0.0, 0.0, 0.0]",0.0,0.0,0.4,0.4,0.4,0.000000,0.0,0.050000


In [21]:
df['tsSkewScore'] = 1.0 - abs(df['tsSkew'])
df['tsMadmScore'] = 1.0 - df['tsMadm']/30.0
df['tsMadmScore'] = df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
df['tsConnCountScore'] = (df['conn_count']) / df['tsConnDiv']
df['tsConnCountScore'] = df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
df['tsScore'] = (((df['tsSkewScore'] + df['tsMadmScore'] + df['tsConnCountScore']) / 3.0) * 1000) / 1000
df.sort_values(by= 'tsScore', ascending=False, inplace=True, ignore_index=True)
df[columns_to_display].head(30)

,tsScore,conn_count,source.ip,destination.ip,virtual_host,http.method,destination.port,deltas
0,1.0,3,192.168.128.7,8.252.196.126,ctldl.windowsupdate.com,GET,80,"[0.0, 0.0]"
1,1.0,4,192.168.1.154,8.240.198.254,3.au.download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0]"
2,1.0,56,192.168.128.50,8.240.196.126,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,1.0,12,192.168.128.50,8.240.200.126,au.download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,1.0,4,192.168.128.50,8.240.225.126,au.download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0]"
5,1.0,21,192.168.1.154,8.240.217.126,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,1.0,3,192.168.128.11,104.84.231.155,download.windowsupdate.com,GET,80,"[0.0, 0.0]"
7,1.0,16,192.168.128.50,8.240.226.254,au.download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,1.0,17,192.168.128.11,104.84.231.160,au.download.windowsupdate.com,GET,80,"[0.0, 755.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,1.0,166,192.168.128.11,104.84.231.160,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5584.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"


In [22]:
df.loc[df['tsScore'] > 0.80, columns_to_display]

,tsScore,conn_count,source.ip,destination.ip,virtual_host,http.method,destination.port,deltas
0,1.000000,3,192.168.128.7,8.252.196.126,ctldl.windowsupdate.com,GET,80,"[0.0, 0.0]"
1,1.000000,4,192.168.1.154,8.240.198.254,3.au.download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0]"
2,1.000000,56,192.168.128.50,8.240.196.126,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,1.000000,12,192.168.128.50,8.240.200.126,au.download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,1.000000,4,192.168.128.50,8.240.225.126,au.download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...
125,0.877592,128,192.168.128.25,69.28.165.128,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12114.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
126,0.866667,3,192.168.128.11,104.100.156.12,crl.disa.mil,GET,80,"[24.0, 0.0]"
127,0.862264,6,192.168.1.154,8.252.195.126,msedge.b.tlu.dl.delivery.mp.microsoft.com,GET,80,"[16.0, 1.0, 3.0, 55.0, 12.0]"
128,0.846032,13,192.168.1.154,23.66.101.157,2.au.download.windowsupdate.com,GET,80,"[2.0, 2.0, 1.0, 2.0, 4.0, 0.0, 8.0, 4.0, 1.0, 5.0, 2.0, 3.0]"
